In [63]:
# Scraping song lyrics from Genius and Spotify
# Spotify client ID: 2573a6dc58484146b0434289ffe03af5
# Spotify client Secret: 78ca0bfc6e77486991c3faa1d507afc2
# user_id = this will probably be a list, will test with https://open.spotify.com/user/spotify?si=3991b93174d04be3
# playlist_id = also a list, testing with https://open.spotify.com/playlist/37i9dQZF1E35BhKQRss82L
# This particular class scrapes a playlist for lyrics, will need to find a way to extrapolate this out to multiple playlists/ an entire category. May need to look at creating a list of user/playlist IDs to loop over? 
# Will use this class if I have time, building model first with Kaggle data. 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import json

spotify_client_id = '2573a6dc58484146b0434289ffe03af5'
spotify_client_secret = '78ca0bfc6e77486991c3faa1d507afc2'
user_id = '3991b93174d04be3'
playlist_id = '37i9dQZF1E35BhKQRss82L'

#a class to scrape Genius HTML for Lyrics using Spotify artist/track information
class GetSpotifyInfo():
    
    #initializing class
    def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        self.user_id = user_id
        self.playlist_id = playlist_id

    #function to use Spotify credentials to get playlist tracks using API credentials and user/playlist ID    
    def get_playlist_info(self):
        credentials = SpotifyClientCredentials(client_id = self.spotify_client_id, client_secret = self.spotify_client_secret)
        sp = spotipy.Spotify(client_credentials_manager=credentials)
        playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    #appends track names into a list
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        print(*track_names)
        self.track_names = track_names
        return self.track_names
    
    #appends artist names into a list
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists

    #get track id, feed this list into audio features function to get audio features for this playlist.
    def get_track_id(self):
        track_id = []
        for song in range(len(self.playlist['items'])):
            track_id.append(self.playlist['items'][song]['track']['id'])
        self.track_id = track_id
        return self.track_id

In [59]:
spotify_song_features = pd.read_csv("../spotify_songs.csv")
spotify_song_features.columns

Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'language'],
      dtype='object')

In [55]:
lyricscsv = pd.read_csv("../Spotify-Playlist/lyrics-data.csv")
lyrics_csv = lyricscsv.drop(columns=['ALink', 'SLink'])
lyricscsv

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [48]:
# importing fasttext to explore the model here
import fasttext
pretrained_model_path = "../fastText/lid.176.bin"
model = fasttext.load_model(pretrained_model_path)

In [51]:
sentence = ['Tudo o que eu quero nessa vida', 'Sweat in the heat for days on end', 'Je suis un chien']
model.predict(sentence)

([['__label__pt'], ['__label__en'], ['__label__fr']],
 [array([0.9250701], dtype=float32),
  array([0.8138323], dtype=float32),
  array([0.9999173], dtype=float32)])